# Constant values
lat = 71.323
lon = -156.609
alt = 8m

# Moving the ARM solar data from the netcdf originator files to csv files for further process
##Current file is nsaarmbecldradC1.c1.20060101.000000.cdf
This file contain hourly data covering the period of a year.
Out of it I am taking the following variables.
tot_cld : Total Cloud Fraction based on MMCR/MPL (satille names), narrow field-of-view, hourly mean
swdn : Surface Downwelling Shortwave Hemispheric Irradiance, best estimate, hourly mean
pmv : Precipitable water vapor best-estimate value, hourly mean
lwp : Liquid water path best-estimate value, hourly mean



All of the following variables names end in the satillie that the data originates from

TSI, ~30s data
tot_cld_tsi : Total Cloud Fraction, from Total Sky Imager, 100 degree FOV,  hourly mean

GOES, NASA - Langley VISST product, 30min data
cld_low_sat_VISST : Satellite-measured low level cloud, clouds below 2 km
cld_mid_sat_VISST : Satellite-measured middle level cloud, clouds between 2 and 6 km
cld_high_sat_VISST : Satellite-measured high level cloud, clouds above 6 km
tot_cld_sat_VISST : Satellite-measured total cloud
cld_thick_sat_VISST : Satellite-measured cloud thickness
cld_top_sat_VISST : Satellite-measured cloud top

Terra & Aqua, NASA-Langley CRS product, 1x1 degree data
cld_low_sat_CERES : Low level clouds, satellite-measured, clouds below 2 km
cld_mid_sat_CERES : Middle level clouds, satellite-measured, clouds between 2 and 6 km (including 2 and 6 km)
cld_high_sat_CERES : High level clouds, satellite-measured, clouds above 6 km
tot_cld_sat_CERES : Total cloud,satellite-measured

The following data is two dimensional and had to be removed when importing from netcdf 
for usablity. I also did not think that having the increaed amount of data would be helpful
for processing time. 

cld_frac
cld_frac_MMCR
cld_frac_MPL
qc_cld_frac
base_time
time_offset
time_bounds

### New note: I have just noticed that I have a limit of 6 years of data for these files. The other files have the full 10 years of data

In [ ]:
# To export to csv and join the data together
import pandas as pd
# Used to import the netcdf file. Removed uneeded data. Then transfer data to pd.DataFrame
import xarray as xr
file = '/home/nelson/PycharmProjects/Solar Forecasting Thesis Project/Data_Preperation/test exploration/nsaarmbecldradC1.c1.20060101.000000.cdf'
ds = xr.open_dataset(file, drop_variables=["cld_frac", "cld_frac_MMCR", "cld_frac_MPL",
                                             "qc_cld_frac", "time_bounds", "time_offset",
                                             "time_bounds"])
df = ds.to_dataframe()

##Current file is nsaarmbecldradC1.c1.20060101.003000.nc
This data is simaliar the the above data. Main differences are that the cloud data is not
seperated by satillite type. It also has the skycover variable.

skycover : Fractional sky cover, based on shortwave flux, hourly mean. The angular portion of the sky 
view that contains clouds divided by the total angular hemispheric view, which is estimated
 from the ARM broadband SW radiometer measurements using the Hemispheric Sky Imager (HIS) and 
 diffuse cloud effect regression relationship described in Long et al.
  1999 (http://science.arm.gov/~clong/SkyCov_art/1999radC.htm).


For some reason the time stamps are not converting over. So I have to convert the 
base time to something useable. Which means translating the int that base time is in to seconds
where the date is seconds since 1970-1-1 0:00:00 0:00.

In [ ]:
file2 = "/home/nelson/PycharmProjects/Solar Forecasting Thesis Project/Data_Preperation/test exploration/nsaarmbecldradC1.c1.20060101.003000.nc"
# variables are dropped if they are not needed to save space and avoid 2 dimensional data
ds2 = xr.open_dataset(file2, drop_variables=["cld_frac", "cld_frac_MPL", "cld_base_source_status",
                                             "qc_tot_cld", "swdif"
                                             "qc_cld_frac", "time_bounds", "time_frac",
                                             "source_cld_frac", "height", "stdev_sedif"
                                            "cld_frac_radar", "time", "qc_swdif",
                                             "swdir", "stdev_swdir", "qc_swdir",
                                             "swup", "stdev_swup", "qc_swup",
                                             "lwdn", "stdev_lwdn", "qc_lwdn",
                                             "lwup", "stdev_lwup", "qc_lwup",
                                             "lw_net_TOA",
                                             "sw_net_TOA", "sw_dn_TOA", "totswfluxdn",
                                             "qc_totswfluxdn", "stdev_totswfluxdn", "alt",
                                             "lat", "lon"], decode_times=False)
df2 = ds2.to_dataframe()

# Add base time (seconds since 1970-1-1 0:00:00 0:00) which in 2006 Jan 1st in this case
# and the time offset. This will get me a useable timestamp in time_offset that 
# I can make into a datetime 
df2["time_offset"] = df2["time_offset"] + df2["base_time"]
df2["time_offset"] = pd.to_datetime(df2["time_offset"], unit='s')

# Dropping all unneeded columns from the DataFrame to save data
drop_cols = ["base_time"]
df2.drop(drop_cols, axis="columns")

## Joining the data together for both of the above files
This will work for both of the file types shown above.
I have just done this for one of the files though to simplify the example.


In [ ]:
# make a small test sample which in this case is another instance of the above df
df1j = ds.to_dataframe()
df_test = pd.concat([df, df1j])
# show the shape of original dataframe
print(df.shape)
# show the shape after joining
print(df_test.shape)

